In [1]:
from gensim.models.word2vec import Word2Vec
#导入相关的package
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
#导入一系列包
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
file_path2='D:/files/data/stopwordlisteng.TXT'
stopworddic1=stop_words
data=[]
with open(file_path2, "r") as f:
    for line in f.readlines():
        line = line.strip('\n')  #去掉列表中每一个元素的换行符
        data.append(line)
        #print(line)
for i in range(len(data)):
    stopworddic1.append(data[i])

fh=[',','(',')','e','.','g','-']

for i in range(1000):
    fh.append(str(i))

for i in range(len(fh)):
    stopworddic1.append(fh[i])

In [5]:
# Import Dataset
file_path='C:/Users/asus/Desktop/w2v-train.xls'
df=pd.read_excel(file_path)
data = df.Content.values.tolist()

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [8]:
stop_words = stopworddic1
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
data_words
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [10]:
df1=df
for j in range(len(df1)):
    str1=''
    for i in range(len(data_lemmatized[j])):
        str1=str1+data_lemmatized[j][i]+' '
    df1.loc[j,'Content']=str1+"\n"

In [11]:
#file_path='C:/Users/asus/Desktop/d2v-train.xls'
#df=pd.read_excel(file_path)
df=df1

In [12]:
a=''

In [13]:
for i in range(len(df)):
    a=a+str(df.loc[i,'Content'])

In [14]:
#from gensim.models.word2vec import Word2Vec

In [15]:
#

In [16]:
#File = open("C:/Users/asus/Desktop/w2vtrain1.txt", "w")
#str = ('123456')
fh = open('C:/Users/asus/Desktop/w2vtrain3.txt', 'w',encoding='UTF-8')
fh.write(a)
fh.close()

In [17]:
#from gensim.models.word2vec import Word2Vec

In [18]:
class MySentences(object):
    def __init__(self, fname):
        self.fname = fname
 
    def __iter__(self):
        for line in open(self.fname,'r'):
            yield line.split()

In [19]:
def w2vTrain(f_input, model_output):         
    sentences = MySentences(f_input)
    w2v_model = Word2Vec(sentences, 
                                  min_count = MIN_COUNT, 
                                  workers = CPU_NUM, 
                                  size = VEC_SIZE,
                                  window = CONTEXT_WINDOW
                                 )
    w2v_model.save(ModelDir+model_output)

In [20]:
ModelDir = "C:/Users/asus/Desktop/"
MIN_COUNT = 1
CPU_NUM = 2 # 需要预先安装 Cython 以支持并行
VEC_SIZE = 100
CONTEXT_WINDOW = 5 # 提取目标词上下文距离最长5个词

f_input = "C:/Users/asus/Desktop/w2vtrainaa.txt"
model_output = "test_w2v_model"

w2vTrain(f_input, model_output)

In [21]:
#w2v_model = Word2Vec.load(ModelDir + model_output)

#w2v_model.most_similar('cars') # 结果一般


In [22]:
from nltk.corpus import stopwords

StopWords = []

In [23]:
def w2vTrain_removeStopWords(f_input, model_output):         
    sentences = list(MySentences(f_input))
    for idx,sentence in enumerate(sentences):
        sentence = [w for w in sentence if w not in StopWords]
        sentences[idx]=sentence
    w2v_model = Word2Vec(sentences, min_count = MIN_COUNT, 
                                  workers = CPU_NUM, size = VEC_SIZE)
    w2v_model.save(ModelDir+model_output)

w2vTrain_removeStopWords(f_input, model_output)
w2v_model = Word2Vec.load(ModelDir+model_output)


In [24]:
w2v_model.most_similar('car') 

[('driverless', 0.9328546524047852),
 ('airplane_drone', 0.8454441428184509),
 ('claim', 0.8288769721984863),
 ('truck', 0.8131569623947144),
 ('lorry', 0.7844126224517822),
 ('drone', 0.771079421043396),
 ('bus', 0.7682951092720032),
 ('robotic', 0.7663077116012573),
 ('motorcycle', 0.7651386260986328),
 ('robot', 0.7628856897354126)]

In [25]:
print(w2v_model['path'])  

[ 1.106571   -1.2226037   1.4155643  -0.63477594 -0.16478749  0.3674199
 -0.25418895 -0.6997259  -0.8491447   0.4015412  -0.46228647  2.529308
  0.79341465  0.81274104 -0.61908    -0.48822916 -0.17628443 -0.13644104
  0.4551251   1.472375    0.12927724 -0.3597522  -1.1434047  -0.31089604
 -1.9768848   0.7447583   0.69713926  1.0471809  -0.8874582   0.41393065
 -0.89354116 -1.3748914   0.08895781  0.22122662 -0.25137404  0.612911
 -1.4525938   0.16270506  0.45208737  0.73530084  1.0274056   0.2951017
 -0.36206585 -1.7779925  -0.96181077  0.5604727  -0.9026642   0.3985239
  0.38568515  1.4399769  -0.00440497 -1.2084484   0.17095461 -0.00947508
  0.34211367 -0.48651433  0.44594568 -1.7749926  -0.4388882  -1.8592781
 -1.566194    0.19203897  1.4153181   0.25827578  1.0019852  -0.83768666
  0.75215125 -1.8311218  -1.506032   -0.3058403  -0.02231756 -0.6331607
 -0.68986416 -0.9262167   0.05435105 -0.8348888   0.14143947  0.45260462
  0.33126163  0.53320783  0.2869112   1.1032283  -0.80047315

In [26]:
a=w2v_model['driver']
a

array([-0.3564262 ,  0.59579676,  0.04229829, -0.15330373,  0.6934745 ,
        0.7479591 , -0.29999715, -0.1007083 , -0.5592124 , -0.2851319 ,
        1.5171393 , -0.7095642 , -0.1790874 , -0.7267299 ,  0.9370492 ,
        0.52035534,  0.04630195,  0.7749454 ,  1.0547695 , -0.73216057,
       -0.04130465,  0.282193  , -0.2676668 ,  0.44541496, -1.8769599 ,
        0.39681017, -0.7157138 , -1.4489734 , -0.92245203, -1.5997937 ,
        0.09262718,  0.10508811, -1.0392758 , -0.8886248 , -1.4584877 ,
        0.24967322, -1.8322527 ,  1.0816203 ,  0.25602812,  0.6951548 ,
        0.68942165,  0.63986444, -0.8763744 , -0.3907447 ,  0.41113982,
       -0.88204974, -0.33869898, -0.50561124, -0.34973282,  0.1028058 ,
       -0.23361997,  0.1891805 ,  0.9026714 , -0.7196632 , -0.54569674,
        0.06497657,  0.14687973, -0.6375202 , -0.46571007, -1.840155  ,
       -0.8164577 , -0.87022316,  0.339395  , -0.0513829 , -1.8103609 ,
        0.8685439 , -0.49754286, -0.23521738, -0.39148545,  1.27

In [27]:
b=1.8*a+8.2*a

In [28]:
b

array([ -3.5642622 ,   5.9579678 ,   0.4229829 ,  -1.5330373 ,
         6.934745  ,   7.4795904 ,  -2.9999714 ,  -1.0070829 ,
        -5.5921235 ,  -2.851319  ,  15.171392  ,  -7.095642  ,
        -1.790874  ,  -7.2672987 ,   9.370492  ,   5.2035537 ,
         0.46301952,   7.7494535 ,  10.547695  ,  -7.3216057 ,
        -0.41304648,   2.82193   ,  -2.6766677 ,   4.4541497 ,
       -18.7696    ,   3.9681015 ,  -7.157138  , -14.489734  ,
        -9.22452   , -15.997936  ,   0.9262718 ,   1.050881  ,
       -10.392757  ,  -8.886248  , -14.584877  ,   2.4967322 ,
       -18.322527  ,  10.816203  ,   2.5602813 ,   6.9515476 ,
         6.8942165 ,   6.398644  ,  -8.763744  ,  -3.9074469 ,
         4.111398  ,  -8.8204975 ,  -3.3869898 ,  -5.0561123 ,
        -3.497328  ,   1.028058  ,  -2.3361998 ,   1.8918049 ,
         9.026713  ,  -7.1966314 ,  -5.4569674 ,   0.64976573,
         1.4687973 ,  -6.3752017 ,  -4.6571007 , -18.40155   ,
        -8.164577  ,  -8.702231  ,   3.3939497 ,  -0.51

In [29]:
str="0.047*vehicle+ 0.031*driver+ 0.026*automate+ 0.022*drive+0.018*study+ 0.017*autonomous+ 0.016*car+ 0.014*safety+0.013*human+ 0.012*technology+"

In [30]:
data=[]
danci=[]
r=['0','1','2','3','4','5','6','7','8','9','.']

In [31]:
strdata=''
strdanci=''
for i in range(len(str)):
    if str[i]=='*':
        data.append(strdata)
        strdata=''
    if str[i]=='+':
        danci.append(strdanci)
        strdanci=''
    if str[i] in  r:
        strdata=strdata+str[i]
    if str[i]!='*' and  str[i]!='+' and  str[i] not in  r and str[i]!=' ':   
        strdanci=strdanci+str[i]
    

In [32]:
data
danci

['vehicle',
 'driver',
 'automate',
 'drive',
 'study',
 'autonomous',
 'car',
 'safety',
 'human',
 'technology']

In [33]:
#data1=data[0]

In [34]:
#type(data1)

In [35]:
#float(data1)

In [36]:
file_path='D:/files/data/finaldata/formatwos.xls'
df=pd.read_excel(file_path)

In [37]:
type(df)

pandas.core.frame.DataFrame

In [38]:
formatwos=df.format.tolist()

In [39]:
formatwos[1]

'0.062*control+0.057*vehicle+0.026*autonomous+0.024*model+0.022*base+0.019*propose+0.018*controller+0.018*path+0.017*trajectory+0.015*method+'

In [40]:
def mycalculate(str):
    data=[]
    danci=[]
    res=[]
    r=['0','1','2','3','4','5','6','7','8','9','.']
    strdata=''
    strdanci=''
    for i in range(len(str)):
        if str[i]=='*':
            data.append(strdata)
            strdata=''
        if str[i]=='+':
            danci.append(strdanci)
            strdanci=''
        if str[i] in  r:
            strdata=strdata+str[i]
        if str[i]!='*' and  str[i]!='+' and  str[i] not in  r:   
            strdanci=strdanci+str[i]
    res.append(data)
    res.append(danci)
    return res
        

In [41]:
mydata=[]
mydanci=[]
for i in range(len(formatwos)):
    str=formatwos[i]
    res1=mycalculate(str)
    mydata.append(res1[0])
    mydanci.append(res1[1])
    

In [42]:
#mydata

In [43]:
myres=[]

In [44]:
for i in range(len(mydata)):
    res=0
    a1=mydata[i]
    a2=mydanci[i]
    for j in range(len(a1)):
        res=res+float(a1[j])*w2v_model[a2[j]]
    myres.append(res)
        
               
    

In [45]:
#print(w2v_model['path']) 
myres

[array([ 2.57536843e-02, -3.33493343e-04,  1.33055970e-01,  4.33415640e-03,
         6.36490658e-02,  1.20933197e-01, -1.38264716e-01, -1.03749052e-01,
        -5.26593253e-02,  1.05524827e-02,  2.16423959e-01,  1.34158805e-01,
         2.56134160e-02,  8.67092051e-03,  9.60598290e-02, -4.94499505e-03,
         3.71092670e-02,  4.78508025e-02,  3.15951854e-02,  6.06562383e-02,
        -9.16355327e-02,  8.94622654e-02,  2.14158073e-02, -5.25471754e-02,
        -2.04733089e-01,  7.97666907e-02, -4.82865833e-02, -2.49555394e-01,
        -1.48537740e-01, -1.41005322e-01,  1.20974734e-01,  6.99909627e-02,
         1.35056255e-03, -7.50726089e-02, -2.06101224e-01, -4.34351619e-03,
        -3.10183913e-01,  1.67483732e-01,  2.80604186e-03,  7.84537122e-02,
         1.54352695e-01,  9.43784565e-02, -1.13564827e-01, -1.62806094e-01,
         9.43678692e-02, -1.03403039e-01, -3.76149192e-02, -8.15536901e-02,
        -5.15578547e-03,  7.54663348e-02,  7.22064986e-04, -6.12421408e-02,
         5.6

In [46]:
file_path='D:/files/data/finaldata/formatdii.xls'
df=pd.read_excel(file_path)

In [47]:
formatdii=df.format.tolist()

In [48]:
mydata=[]
mydanci=[]
for i in range(len(formatdii)):
    str=formatdii[i]
    res1=mycalculate(str)
    mydata.append(res1[0])
    mydanci.append(res1[1])
   

In [49]:
myres1=[]

In [50]:
for i in range(len(mydata)):
    res=0
    a1=mydata[i]
    a2=mydanci[i]
    for j in range(len(a1)):
        res=res+float(a1[j])*w2v_model[a2[j]]
    myres1.append(res)

In [51]:
myres

[array([ 2.57536843e-02, -3.33493343e-04,  1.33055970e-01,  4.33415640e-03,
         6.36490658e-02,  1.20933197e-01, -1.38264716e-01, -1.03749052e-01,
        -5.26593253e-02,  1.05524827e-02,  2.16423959e-01,  1.34158805e-01,
         2.56134160e-02,  8.67092051e-03,  9.60598290e-02, -4.94499505e-03,
         3.71092670e-02,  4.78508025e-02,  3.15951854e-02,  6.06562383e-02,
        -9.16355327e-02,  8.94622654e-02,  2.14158073e-02, -5.25471754e-02,
        -2.04733089e-01,  7.97666907e-02, -4.82865833e-02, -2.49555394e-01,
        -1.48537740e-01, -1.41005322e-01,  1.20974734e-01,  6.99909627e-02,
         1.35056255e-03, -7.50726089e-02, -2.06101224e-01, -4.34351619e-03,
        -3.10183913e-01,  1.67483732e-01,  2.80604186e-03,  7.84537122e-02,
         1.54352695e-01,  9.43784565e-02, -1.13564827e-01, -1.62806094e-01,
         9.43678692e-02, -1.03403039e-01, -3.76149192e-02, -8.15536901e-02,
        -5.15578547e-03,  7.54663348e-02,  7.22064986e-04, -6.12421408e-02,
         5.6

In [52]:
myres1[1][0]

0.17673612

In [53]:
vector1 =myres[0]
vector2 = myres1[2]
dist = np.sqrt(np.sum(np.square(vector1 - vector2 )))
dist
#num=float(numpy.sum(vector1*vector2))

2.244646

In [54]:
num=float(np.sum(vector1*vector2))
denom=np.linalg.norm(vector1)*np.linalg.norm(vector2)
cos=num/denom
cos

0.8324243887827353

In [55]:
dd1=[]
for i in range(len(myres)):
    kkk = []
    for j in range(len(myres1)):
        vector1 =myres[i]
        vector2 = myres1[j]
        num=float(np.sum(vector1*vector2))
        denom=np.linalg.norm(vector1)*np.linalg.norm(vector2)
        cos=num/denom
        #dist = np.sqrt(np.sum(np.square(vector1 - vector2 )))
        kkk.append(cos)
    dd1.append(kkk) 

In [56]:
dd

NameError: name 'dd' is not defined

In [ ]:
dd1

In [57]:
dd2=[]
for i in range(len(myres)):
    kkk = []
    for j in range(len(myres)):
        vector1 =myres[i]
        vector2 = myres[j]
        num=float(np.sum(vector1*vector2))
        denom=np.linalg.norm(vector1)*np.linalg.norm(vector2)
        cos=num/denom
        #dist = np.sqrt(np.sum(np.square(vector1 - vector2 )))
        kkk.append(cos)
    dd2.append(kkk) 

In [58]:
dd3=[]
for i in range(len(myres1)):
    kkk = []
    for j in range(len(myres1)):
        vector1 =myres1[i]
        vector2 = myres1[j]
        num=float(np.sum(vector1*vector2))
        denom=np.linalg.norm(vector1)*np.linalg.norm(vector2)
        cos=num/denom
        #dist = np.sqrt(np.sum(np.square(vector1 - vector2 )))
        kkk.append(cos)
    dd3.append(kkk) 

In [59]:
dd2

[[1.0,
  0.7289841348502514,
  0.5390240965388275,
  0.8050332642123517,
  0.7443484124243588,
  0.41825181221634466,
  0.7996290291520088,
  0.7624174050641229,
  0.7834087057400029],
 [0.7289841348502514,
  1.000000096770844,
  0.7953939824527846,
  0.694281870878825,
  0.8431021937314349,
  0.5611317260144962,
  0.7370736386029821,
  0.6548276404287796,
  0.8759393884993183],
 [0.5390240965388275,
  0.7953939824527846,
  1.0000000735040744,
  0.6152495675869347,
  0.689007249908193,
  0.7673593491629453,
  0.5574222446625403,
  0.6100850987597529,
  0.6569751716190545],
 [0.8050332642123517,
  0.694281870878825,
  0.6152495675869347,
  0.9999999361259739,
  0.6121422913226059,
  0.5626508416771604,
  0.7758698751836152,
  0.7735851297690332,
  0.7125086255961585],
 [0.7443484124243588,
  0.8431021937314349,
  0.689007249908193,
  0.6121422913226059,
  1.0,
  0.668293306455756,
  0.7905144177974691,
  0.6168318060224958,
  0.9417382899875693],
 [0.41825181221634466,
  0.5611317260144

In [60]:
dd3

[[1.0,
  0.8585141757913639,
  0.8132434728574118,
  0.3528661903469807,
  0.6480776377727712,
  0.7253854588471191,
  0.4314745113950524,
  0.8000854871098924,
  0.8483316142032564,
  0.6215837570971469,
  0.5427356260218562,
  0.7881765795588415],
 [0.8585141757913639,
  1.0,
  0.7798529848643866,
  0.3209443184061978,
  0.6085794833875148,
  0.6888607043098789,
  0.26591999258791227,
  0.7031290709791824,
  0.7344789696143571,
  0.515426690802212,
  0.4231418500091626,
  0.7669565429816249],
 [0.8132434728574118,
  0.7798529848643866,
  1.0000000971175038,
  0.515458205734995,
  0.8716245894680605,
  0.8005954316484603,
  0.40903559262061695,
  0.8334098708821455,
  0.7281164588152502,
  0.46862757180185194,
  0.7367516753557969,
  0.838596773400387],
 [0.3528661903469807,
  0.3209443184061978,
  0.515458205734995,
  1.0000001136821466,
  0.5170988722831168,
  0.4618329492495529,
  0.7296621000230056,
  0.6852284697911676,
  0.4130838229408827,
  0.4679710195279551,
  0.689404099230